In [1]:
import dash
from dash import html, dcc, Input, Output, Dash
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
from datetime import date

# Load Data
data = pd.read_csv("https://raw.githubusercontent.com/00Dabide/Dataviz2_Assignment/main/Coffee%20Shop%20Sales.csv")

# Creating a unique list for the Locations
Store = data.store_location.unique()
Store.sort()

# Create a list for the mode
Mode = ["Revenue", "Quantity"]
# Dictionary interpreter for the mode
ModeMeaning = {"Revenue" : "unit_price", "Quantity" : "transaction_qty"}

App = Dash(__name__)

# Defining the layout
App.layout = (
    html.Div([
        # Header Text
        html.H1("Store sales", style = {"fontSize" : 64, "textAlign" : "center"}),
        
        # 1st Selectable: Location
        html.Div([
            html.Label("Select the location: ", style = {"fontSize" : 28}),
            dcc.Dropdown(id = "Location", options = Store, value = "Astoria", style = {"width" : "40%"})]),

        # 2nd Selectable: Date
        html.Div([
            dbc.Row([html.Label("Select date range: ", style = {"fontSize" : 28})]),
            dbc.Row([dcc.DatePickerRange(id = "Date", min_date_allowed = date(2023,1,1), max_date_allowed = date(2023,12,31), start_date = date(2023,1,1), end_date = date(2023,12,31))])
            ]),

        # 3rd Selectable: Mode (Revenue or Quantity Sold)
        html.Div([
            dbc.Row([html.Label("Select mode: ", style = {"fontSize" : 28})]),
            dbc.Row([dcc.RadioItems(id = "Mode", options = Mode, value = "Revenue", inline = True)])
            ]),
        # The Graph!!!
        dcc.Graph(id = "graph")
]))

# Defining callback
@App.callback(
    Output("graph","figure"),
    Input("Location", "value"),
    Input('Date', 'start_date'),
    Input('Date', 'end_date'),
    Input("Mode", "value"))

def GetFig(Location,start_date,end_date, Mode):
    #Load Data
    data = pd.read_csv("https://raw.githubusercontent.com/00Dabide/Dataviz2_Assignment/main/Coffee%20Shop%20Sales.csv")

    # Convert to date
    data["transaction_date"] = pd.to_datetime(data["transaction_date"])

    # Sort the table as it was previously selected
    data = data.query("store_location == @Location and transaction_date.between(@start_date, @end_date)")

    # Define the Graph
    Fig = px.histogram(data, x = "product_category"	, y = ModeMeaning[Mode],
                   title = "Sales",
                   text_auto = True,
                   labels = {"x" : "Category",
                             "y" : "Revenue"},
                   template = "plotly_dark",
                   color_discrete_sequence=['white'],
                   height = 400,
                   width = 1200)
    Fig.update_layout(xaxis_title = "Category",
                      yaxis_title = Mode)
    return Fig

App.run()

'unit_price'